In [1]:
import sympy as sp
from sympy.codegen.cfunctions import log10
from IPython.display import display, Markdown

In [2]:
def eliminate(eqs, z):
    """return eqs with parameter z eliminated from each equation; the first
    element in the returned list will be the definition of z that was used
    to eliminate z from the other equations.

    Examples
    ========

    >>> eqs = [Eq(2*x + 3*y + 4*z, 1),
    ...        Eq(9*x + 8*y + 7*z, 2)]
    >>> eliminate(eqs, z)
    [Eq(z, -x/2 - 3*y/4 + 1/4), Eq(11*x/2 + 11*y/4 + 7/4, 2)]
    >>> Eq(y,solve(_[1], y)[0])
    Eq(y, -2*x + 1/11)
    """
    from sympy.solvers.solveset import linsolve
    Z = sp.Dummy()
    rv = []
    for i, e in enumerate(eqs):
        if z not in e.free_symbols:
            continue
        e = e.subs(z, Z)
        if z in e.free_symbols:
            break
        try:
            s = linsolve([e], Z)
            if s:
                zi = list(s)[0][0]
                rv.append(sp.Eq(z, zi))
                rv.extend([eqs[j].subs(z, zi)
                    for j in range(len(eqs)) if j != i])
                return rv
        except ValueError:
            continue
    raise ValueError('only a linear parameter can be eliminated')

### Coupled Equilibria
---
$\require{mhchem}$
$
\ce{G <=> H_2A} \\
\ce{H_2A <=> H^+ + HA^-} \\
\ce{HA^- <=> H^+ + A^2-}
$
---
$
    \rm
    K_H = \frac{[H_{2}A]}{p_G} \\
    K_1 = \frac{[H^+][HA^-]}{[H_{2}A]} \\
    K_2 = \frac{[H^+][A^{2-}]}{[H_{2}A]}
$

In [3]:
K_H, K_1, K_2 = sp.symbols('K_H K_1 K_2')
n_H, c_1, c_2 = sp.symbols('n_H c_1 c_2')
V_g, V_l, R, T = sp.symbols('V_g V_l R T')
G, H2A, HA, A, H = sp.symbols('p_G [H_{2}A] [HA] [A] [H]')
G_0, H2A_0, HA_0, A_0, H_0 = sp.symbols('p_{G0} [H_{2}A]_0 [HA^{-}]_0 [A^{2-}]_0 [H^{+}]_0')

In [4]:
# List of equilibria
eqs = [
    K_H * G - H2A,
    K_1 * H2A - HA * H,
    K_2 * HA - A * H
]

# List of changes
chgs = {
        G_0: G + n_H * R * T / V_g,
        H2A: H2A_0 + n_H / V_l - c_1,
        HA: HA_0 + c_1 - c_2,
        A: A_0 + c_2,
        H: H_0 + c_1 + c_2
    }

# Elimination and simplification
eqs = [eq.subs(chgs) for eq in eqs]
eqs = eliminate(eqs, n_H)
eqs[1:] = eliminate(eqs[1:], c_1)
# Multiply by divisor for numeric stability
eqs[2] *= (A_0 - K_2 + c_2)**2
eqs[2] = sp.simplify(eqs[2])

In [5]:
display(Markdown('---\n### Solution for Changes\n---'))
for eq in eqs:
    display(eq)
display(Markdown('---\n### Solution for Equilibrium Concentrations\n---'))
for var, chg in chgs.items():
    display(sp.Eq(var, chg))

---
### Solution for Changes
---

Eq(n_H, K_H*V_l*p_G - V_l*[H_{2}A]_0 + V_l*c_1)

Eq(c_1, (-K_2*[HA^{-}]_0 + K_2*c_2 + [A^{2-}]_0*[H^{+}]_0 + [A^{2-}]_0*c_2 + [H^{+}]_0*c_2 + c_2**2)/(K_2 - [A^{2-}]_0 - c_2))

K_1*K_H*p_G*(-K_2 + [A^{2-}]_0 + c_2)**2 + (-K_2*[HA^{-}]_0 + K_2*c_2 + [A^{2-}]_0*[H^{+}]_0 + [A^{2-}]_0*c_2 + [H^{+}]_0*c_2 + c_2**2 + (-[HA^{-}]_0 + c_2)*(-K_2 + [A^{2-}]_0 + c_2))*(K_2*[HA^{-}]_0 - K_2*c_2 - [A^{2-}]_0*[H^{+}]_0 - [A^{2-}]_0*c_2 - [H^{+}]_0*c_2 - c_2**2 + ([H^{+}]_0 + c_2)*(-K_2 + [A^{2-}]_0 + c_2))

---
### Solution for Equilibrium Concentrations
---

Eq(p_{G0}, R*T*n_H/V_g + p_G)

Eq([H_{2}A], [H_{2}A]_0 - c_1 + n_H/V_l)

Eq([HA], [HA^{-}]_0 + c_1 - c_2)

Eq([A], [A^{2-}]_0 + c_2)

Eq([H], [H^{+}]_0 + c_1 + c_2)

In [6]:
# Specify conditions
params = {
    K_H: 1.2e-5, # M / Pa
    K_1: 10**-1.81, # M
    K_2: 10**-7.17, # M
    R: 8314.46261815324, # (N * L) / (mol * m^2 * K)
    T: 298.15, # K
    V_g: 1000, # L
    V_l: 1, # L
    G: 1.013e5, # Pa
    H2A_0: 0, # M
    HA_0: 0, # M
    A_0: 0, # M
    H_0: 1e-7 # M
}
# Select solution of cubic equation (eqs[2])
solution_index = 0

# Solve cubic equation
c_2_sol = [sp.re(sol) for sol in sp.solve(eqs[2].subs(params))]
c_2_sol

[6.76081816423000e-8, 6.76084134365235e-8, 69619.4679673774]

In [7]:
# Get solutions
params.update(c_2=c_2_sol[solution_index])
params.update(c_1=sp.solve(eqs[1].subs(params))[0])
params.update(n_H=sp.solve(eqs[0].subs(params))[0])

print('G_0 =', chgs[G_0].subs(params), 'Pa')
print('H2A =', chgs[H2A].subs(params), 'M')
print('HA =', chgs[HA].subs(params), 'M')
print('A =', chgs[A].subs(params), 'M')
print('pH =', -log10(float(chgs[H].subs(params))))

G_0 = 104653.565239496 Pa
H2A = 1.21560000000000 M
HA = 0.137212909563224 M
A = 6.76081816423000e-8 M
pH = 0.862604281983567
